In [9]:
pip install os

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected symbol
1: pip install
        ^


In [4]:
#set the wd to file location
import os

abspath = os.path.abspath('dataclean/ipynb')
dname = os.path.dirname(abspath)
os.chdir(dname)

ERROR: Error in parse(text = x, srcfile = src): <text>:2:8: unexpected symbol
1: #set the wd to file location
2: import os
          ^


# dataClean File
The purpose of this file is to define a function which cleans the data by removing columns. It may be expanded to increase the scope. It requires the following files:
<ul>
    <li> 01-vehicles-download.csv </li>
</ul>
It will produce the following files
<ul>
    <li> 02-vehicles-cleaned.csv </li>
</ul>
    

In [3]:
#Import Necessary libraries
library('tidyverse')
library('dplyr')
#Read Data
vehicles = read.csv('./01-vehicles-download.csv',header=TRUE)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.5
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message in file(file, "rt"):
“cannot open file './01-vehicles-download.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [1]:
#List of coolumn names to remove


## cleanData Function
The cleanData function takes the following inputs:
<ul>
    <li> df: the dataframe to be cleaned </li>
    <li>  vars: a list of vars to drop </li>
</ul>
The cleanData function creates a new dataframe object which is a copy of the original data frame with the following changes:
<ul>
    <li> it does not contain the columns in the vars list </li>
</ul>

In [1]:
cleanData = function(df,vars){
    cleanedVehicles <<- df[ , !(names(df) %in% vars)]
    return(cleanedVehicles)
    }

## Removing Columns

In [2]:
#applying the cleanData function to the vehicles dataset to filter for variables
filteredVars = c('id','url','region','region_url','VIN','image_url','description','lat','long')

cleanData(vehicles,filteredVars)

ERROR: Error in cleanData(vehicles, filteredVars): object 'vehicles' not found


In [21]:
#save the data
write.csv(cleanedVehicles, '02-vehicles-clean.csv',row.names=FALSE)